# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
import os
import requests
from openai import OpenAI
from dotenv import load_dotenv
from typing import Optional, Literal
from IPython.display import display, Markdown

In [32]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [33]:
# set up environment
load_dotenv()

class Model:
    def __init__(self):
        self.client_oai = OpenAI(api_key=os.getenv("OPENAI_KEY"))
        self.ollama_base_url = f"{os.getenv('OLLAMA_URL')}/api/chat"

    def _prompt_llama(self, text: str):
        response = requests.post(
            self.ollama_base_url,
            json={
                "model": MODEL_LLAMA,
                "messages": [{"role": "user", "content": text}],
                "stream": True
            },
            stream=True
        )
        for line in response.iter_lines():
            if not line:
                continue
                
            data = line.decode("utf-8")
            if data.strip() == "":
                continue
                
            if data.startswith("data:"):
                data = data[5:].strip()
                
            yield data

    def _prompt_oai(self, question: str):
        stream = self.client_oai.chat.completions.create(
            model=MODEL_GPT,
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are an advanced reasoning and explanation engine. "
                        "You write with precision, clarity, and conciseness. "
                        "You can explain Python, algorithms, code design, and systems-level behavior "
                        "with technical rigor, while being straight to the point."
                    ),
                },
                {"role": "user", "content": question},
            ],
            stream=True,
        )
        return stream

    def prompt(self, question: str, model: Optional[Literal[MODEL_GPT, MODEL_LLAMA]] = MODEL_GPT):
        if "gpt" in model:
            stream = self._prompt_oai(question)
            buffer = []
            for event in stream:
                if event.choices and event.choices[0].delta.content:
                    chunk = event.choices[0].delta.content
                    buffer.append(chunk)
                    yield chunk
                    
            output = "".join(buffer)

        else:
            stream = self._prompt_llama(question)
            buffer = []
            for chunk in stream:
                try:
                    import json
                    data = json.loads(chunk)
                    content = data.get("message", {}).get("content", "")
                    
                    if content:
                        buffer.append(content)
                        yield content
                        
                except Exception as e:
                    print("An error occured", e)
                    continue
                    
            output = "".join(buffer)

        display(Markdown(output))

In [25]:
# here is the question; type over this to ask something new
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [27]:
# Get gpt-4o-mini to answer, with streaming
model = Model()

for token in model.prompt(question, model=MODEL_GPT):
    print(token, end="", flush=True)


The provided code snippet is a Python expression that uses `yield from` with a set comprehension. Here’s a breakdown of what it does:

1. **Set Comprehension**: The expression `{book.get("author") for book in books if book.get("author")}` creates a set of unique authors extracted from a list (or any iterable) called `books`. 
    - `book.get("author")` retrieves the value associated with the key `"author"` from each `book` dictionary.
    - The `if book.get("author")` condition filters out any books that do not have an author (i.e., where the author value is `None` or an empty string). 

2. **Yield from**: The `yield from` statement is used in a generator function to yield all values from the iterable that follows it. In this case, it yields the unique authors produced by the set comprehension.

### Why Use This Code?

- **Uniqueness**: Using a set comprehension ensures that only unique authors are collected, automatically eliminating duplicates.
- **Efficiency**: The code succinctly p

The provided code snippet is a Python expression that uses `yield from` with a set comprehension. Here’s a breakdown of what it does:

1. **Set Comprehension**: The expression `{book.get("author") for book in books if book.get("author")}` creates a set of unique authors extracted from a list (or any iterable) called `books`. 
    - `book.get("author")` retrieves the value associated with the key `"author"` from each `book` dictionary.
    - The `if book.get("author")` condition filters out any books that do not have an author (i.e., where the author value is `None` or an empty string). 

2. **Yield from**: The `yield from` statement is used in a generator function to yield all values from the iterable that follows it. In this case, it yields the unique authors produced by the set comprehension.

### Why Use This Code?

- **Uniqueness**: Using a set comprehension ensures that only unique authors are collected, automatically eliminating duplicates.
- **Efficiency**: The code succinctly processes the list of books and yields only the relevant data (authors) directly from the generator, making it memory-efficient and lazy.
- **Readability**: The use of `yield from` keeps the code clean and avoids the need to create an intermediate list before yielding authors.

### Example:

Given a list of books represented as dictionaries:

```python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": "Author A"},
    {"title": "Book 4", "author": None},
]
```

The code would yield `Author A` and `Author B`, iterating over each author only once.

### Conclusion:

The code succinctly generates a lazily yielded sequence of unique authors from a collection of book dictionaries, efficiently handling potential duplicates and missing values.

In [34]:
# Get Llama 3.2 to answer

for token in model.prompt(question, model=MODEL_LLAMA):
    print(token, end="", flush=True)

This line of code is a part of Python's iteration protocol, specifically using the `yield from` keyword.

Here's what it does:

- It generates an iterator that yields values from another iterable.
- The expression `{book.get("author") for book in books if book.get("author")}` generates an iterator over the authors of all books where an author is available (`book.get("author") != None or book.get("author") == ""`).

The `yield from` keyword takes this generator expression and yields from it. It's essentially saying "use this generator to generate values, I'll take them in order".

Here's a step-by-step explanation:

1. `{book.get("author") for book in books if book.get("author")}` generates an iterator over the authors of all books where an author is available.

   - This works by iterating over each `book` in the collection (`books`), checking if it has a valid author, and then yielding the author's name.
   - If `book.get("author")` returns `None`, or if its value is an empty string, 

This line of code is a part of Python's iteration protocol, specifically using the `yield from` keyword.

Here's what it does:

- It generates an iterator that yields values from another iterable.
- The expression `{book.get("author") for book in books if book.get("author")}` generates an iterator over the authors of all books where an author is available (`book.get("author") != None or book.get("author") == ""`).

The `yield from` keyword takes this generator expression and yields from it. It's essentially saying "use this generator to generate values, I'll take them in order".

Here's a step-by-step explanation:

1. `{book.get("author") for book in books if book.get("author")}` generates an iterator over the authors of all books where an author is available.

   - This works by iterating over each `book` in the collection (`books`), checking if it has a valid author, and then yielding the author's name.
   - If `book.get("author")` returns `None`, or if its value is an empty string, it skips that book.

2. `yield from { ... }` takes this generator expression and yields from it.

   - It's like saying "take all values yielded by the inner generator and use them one by one". 

   This makes the code cleaner and easier to read because you don't have to manually call `next(book_generator)` for each book in the collection. You can simply iterate over this new iterator to get all authors.

However, without more context about how these books and their data are structured, it's hard to tell exactly why or when someone would use this line of code specifically. But generally, it's useful for generating values from other iterables while still following a clear sequence.

Here's an example:

```python
books = [
    {"title": "Book 1", "author": None},
    {"title": "Book 2", "author": "Author 1"},
    {"title": "Book 3", "author": ""}
]

for author in yield from {book.get("author") for book in books if book.get("author")}:
    print(author)
```

This would print:

```
Author 1
```